### **Урок 8. Рекуррентные нейронные сети RNN LSTM GRU**
### Задание.
Данные берем отызывы за лето
На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Activation, Masking, SimpleRNN, Dropout, LSTM, GRU, Flatten, MaxPooling1D
# from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df =  pd.read_excel("отзывы за лето.xls")
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [4]:
df.rename(columns={'Content': 'text'}, inplace=True)
df.head()

,Rating,text,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
# Сокращаем количество классов до 2
df = df[df['Rating'] != 3]
df['class'] = (df['Rating'] > 3)*1
df

,Rating,text,Date,class
0,5,It just works!,2017-08-14,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,1
2,5,Отлично все,2017-08-14,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,1
4,5,"Очень удобно, работает быстро.",2017-08-14,1
...,...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,0
20655,5,Ок,2017-06-01,1
20656,4,Доволен,2017-06-01,1
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,0


In [6]:
df['class'].value_counts()

1    16724
0     3024
Name: class, dtype: int64

In [7]:
train, validate, test = np.split(df.sample(frac=1, random_state=42),[int(.6*len(df)), int(.8*len(df))])

In [8]:
train.to_csv('train.csv')
test.to_csv('test.csv')
validate.to_csv('val.csv')

In [9]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

In [10]:
df_train

,Unnamed: 0,Rating,text,Date,class
0,8446,5,В приложении все устраивает .Немного удивляет ...,2017-08-02,1
1,3819,1,Удаляю это бесполезное недоприожение. В интерн...,2017-08-08,0
2,9239,5,Было обновление в поидожение после которого вы...,2017-08-01,1
3,326,5,Сбербанк всегда рядом! Очень удобное приложени...,2017-08-13,1
4,17944,4,"Пока все хорошо, приложение нравится",2017-07-08,1
...,...,...,...,...,...
11843,16363,5,Очень удобно,2017-07-18,1
11844,244,5,"Проблемы с root правами решена, сбербанк онлай...",2017-08-14,1
11845,4470,5,Удобное приложение.,2017-08-07,1
11846,6140,5,Удобная программа,2017-08-05,1


In [11]:
df_test

,Unnamed: 0,Rating,text,Date,class
0,9717,5,Приложение отличное!,2017-07-31,1
1,15801,5,Очень удобно!,2017-07-21,1
2,19041,5,Гуд,2017-06-29,1
3,8863,5,Klass,2017-08-01,1
4,10305,1,После последнего обновления пользоваться стало...,2017-07-30,0
...,...,...,...,...,...
3945,11772,5,Хорошее приложение,2017-07-28,1
3946,12491,5,Хорошо,2017-07-27,1
3947,5634,5,Всё супер круть,2017-08-06,1
3948,901,1,ХРеново.деньги чтобы перечислить надо танцы с ...,2017-08-12,0


In [12]:
df_val

,Unnamed: 0,Rating,text,Date,class
0,102,5,Замечательное приложение,2017-08-14,1
1,13646,5,Good ),2017-07-25,1
2,17559,1,Надо заходить по 2 раза. Пишет что стоит рут п...,2017-07-11,0
3,15986,5,"Отлично, удобно и просто!!!",2017-07-20,1
4,12527,5,"Отличное приложение,без него уже и немогу!!!",2017-07-27,1
...,...,...,...,...,...
3945,4666,5,"Супер,спасибо!!!",2017-08-07,1
3946,1540,5,Отличное приложение! Оч.удобное.,2017-08-11,1
3947,11170,5,Отлично!,2017-07-29,1
3948,15565,5,Отличное приложение. Облегчает жизнь,2017-07-21,1


In [13]:
!pip install pymorphy2
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()

    txt = re.sub("\sне", "не", txt)

    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]

    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [15]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [16]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

x_train = pad_sequences(sequences_train, maxlen=training_length)
x_val = pad_sequences(sequences_val, maxlen=training_length)

In [17]:
y_train = df_train['class'].values
y_val = df_val['class'].values

result = []

## **CNN**

In [18]:
def model_CNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()
  model.add(Embedding(input_dim=word_count, 
                      output_dim=128, 
                      input_length=training_length))
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  result.append(['CNN', loss, accuracy])

  return result

In [19]:
model_CNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 132, 128)          1154304   
                                                                 
 conv1d (Conv1D)             (None, 130, 128)          49280     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,204,885
Trainable params: 1,204,885
Non-trainable params: 0
______________________________________________

[['CNN', 0.27366575598716736, 0.8870885968208313]]

## **RNN**

In [20]:
def model_simple_RNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))

  model.add(SimpleRNN(64))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  result.append(['RNN', loss, accuracy])

  return result

In [21]:
model_simple_RNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 132, 30)           270540    
                                                                 
 masking (Masking)           (None, 132, 30)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6080      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 280,845
Trainable params: 280,845
Non-tr

[['CNN', 0.27366575598716736, 0.8870885968208313],
 ['RNN', 0.28865477442741394, 0.8789873123168945]]

## **LSTM**

In [22]:
def model_LSTM(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))
  model.add(LSTM(128, recurrent_dropout=0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  result.append(['LSTM', loss, accuracy])

  return result

In [23]:
model_LSTM(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 132, 30)           270540    
                                                                 
 masking_1 (Masking)         (None, 132, 30)           0         
                                                                 
 lstm (LSTM)                 (None, 128)               81408     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 360,269
Trainable params: 360,269
Non-tr

[['CNN', 0.27366575598716736, 0.8870885968208313],
 ['RNN', 0.28865477442741394, 0.8789873123168945],
 ['LSTM', 0.30602848529815674, 0.8607594966888428]]

## **GRU**

In [24]:
def model_GRU(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))
  model.add(GRU(128, recurrent_dropout=0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  result.append(['GRU', loss, accuracy])

  return result

In [25]:
model_GRU(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 132, 30)           270540    
                                                                 
 masking_2 (Masking)         (None, 132, 30)           0         
                                                                 
 gru (GRU)                   (None, 128)               61440     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 340,301
Trainable params: 340,301
Non-tr

[['CNN', 0.27366575598716736, 0.8870885968208313],
 ['RNN', 0.28865477442741394, 0.8789873123168945],
 ['LSTM', 0.30602848529815674, 0.8607594966888428],
 ['GRU', 0.2748200297355652, 0.8729113936424255]]

## **CNN -> RNN**

In [26]:
def model_CNN_RNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Conv1D(64, 2, padding='same', activation='relu'))
  model.add(MaxPooling1D(2, padding='same'))
  model.add(Conv1D(32, 2, padding='same', activation='relu'))
  model.add(MaxPooling1D(2, padding='same'))


  model.add(LSTM(64, return_sequences=True, recurrent_dropout = 0.2))
  model.add(LSTM(32))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  result.append(['CNN_RNN', loss, accuracy])

  return result

In [27]:
model_CNN_RNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 132, 30)           270540    
                                                                 
 conv1d_1 (Conv1D)           (None, 132, 64)           3904      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 66, 64)           0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 66, 32)            4128      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 33, 32)           0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 33, 64)           

[['CNN', 0.27366575598716736, 0.8870885968208313],
 ['RNN', 0.28865477442741394, 0.8789873123168945],
 ['LSTM', 0.30602848529815674, 0.8607594966888428],
 ['GRU', 0.2748200297355652, 0.8729113936424255],
 ['CNN_RNN', 0.42638224363327026, 0.845822811126709]]

## **RNN -> CNN**

In [28]:
def model_RNN_CNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = Sequential()

  model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30))
  model.add(LSTM(64, return_sequences=True))
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding="same"))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  result.append(['RNN_CNN', loss, accuracy])

  return result

In [29]:
model_RNN_CNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 132, 30)           270540    
                                                                 
 lstm_3 (LSTM)               (None, 132, 64)           24320     
                                                                 
 conv1d_3 (Conv1D)           (None, 132, 64)           12352     
                                                                 
 flatten (Flatten)           (None, 8448)              0         
                                                                 
 dense_10 (Dense)            (None, 64)                540736    
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                

[['CNN', 0.27366575598716736, 0.8870885968208313],
 ['RNN', 0.28865477442741394, 0.8789873123168945],
 ['LSTM', 0.30602848529815674, 0.8607594966888428],
 ['GRU', 0.2748200297355652, 0.8729113936424255],
 ['CNN_RNN', 0.42638224363327026, 0.845822811126709],
 ['RNN_CNN', 0.3162797689437866, 0.8718987107276917]]

In [30]:
pd.DataFrame(result, columns=['model', 'loss', 'accuracy']).sort_values('accuracy',ascending=False)

,model,loss,accuracy
0,CNN,0.273666,0.887089
1,RNN,0.288655,0.878987
3,GRU,0.274820,0.872911
5,RNN_CNN,0.316280,0.871899
2,LSTM,0.306028,0.860759
4,CNN_RNN,0.426382,0.845823


### **Вывод:** Наилучший результат показала модель CNN